# Hierarchical Clustering (library)
Run in Colab. Uses scipy for dendrogram and sklearn AgglomerativeClustering.

In [ ]:
# Install
!pip install -q matplotlib seaborn scikit-learn scipy

In [ ]:
import numpy as np, matplotlib.pyplot as plt, seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
sns.set(style='whitegrid')
X, y = make_blobs(n_samples=200, centers=4, cluster_std=0.7, random_state=42)

In [ ]:
Z = linkage(X, method='ward')
plt.figure(figsize=(10,4))
dendrogram(Z, truncate_mode='lastp', p=20, leaf_rotation=45.)
plt.title('Dendrogram (truncated)')
plt.show()

In [ ]:
model = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels = model.fit_predict(X)
plt.scatter(X[:,0], X[:,1], c=labels, cmap='tab10', s=10)
plt.title('Agglomerative clustering (Ward)')
plt.show()

## Notes
- Experiment with linkage: single, complete, average.
- Try different distance metrics.
- Record a video walkthrough with dendrogram and scatter plot.